In [4]:
import serial
import csv
import os
from datetime import datetime, timedelta
import time

In [6]:

def reset_esp32(port, baudrate=921600):
    """Resets an ESP32 using the RTS and DTR lines of the serial port.

    Args:
        port: The serial port (e.g., "COM3" on Windows, "/dev/ttyUSB0" on Linux).
        baudrate: The baud rate (typically 115200 for communication,
                  but can be higher for flashing).
    """
    try:
        with serial.Serial(port, baudrate) as ser:
            # Method 1 (esptool.py compatible):
            ser.dtr = False  # DTR = 0: Set EN (reset) to HIGH
            ser.rts = True   # RTS = 1, Set IO0 (GPIO0) to LOW: BOOT mode
            time.sleep(0.1)
            ser.dtr = True   # DTR = 1, Set EN to LOW: Reset chip
            time.sleep(0.1)  # Wait for the chip to reset
            ser.dtr = False  # DTR = 0, set EN to HIGH
            ser.rts = False # Set IO0 to HIGH, back to normal mode.
            time.sleep(0.1)

            # Method 2 (Simple DTR toggle - might work in some cases):
            # ser.dtr = True   # Assert DTR (often connected to reset)
            # time.sleep(0.1)
            # ser.dtr = False  # De-assert DTR
            # time.sleep(0.1)
            print(f"ESP32 reset via DTR/RTS on port {port}")

    except serial.SerialException as e:
        print(f"Error resetting ESP32: {e}")


In [19]:
def collect_csi_data(filepath, port, baudrate = 921600, seconds = 100, minutes = 0, hours = 0):
    with serial.Serial(port, baudrate) as ser:
        header = "type,seq,mac,rssi,rate,sig_mode,mcs,bandwidth,smoothing,not_sounding,aggregation,stbc,fec_coding,sgi,noise_floor,ampdu_cnt,channel,secondary_channel,local_timestamp,ant,sig_len,rx_state,len,first_word,data,datetime"
        
        start_time = datetime.now()
        
        filename = filepath
        temp_file_path = "temp_csi_data.csv"
        
        if not os.path.exists(filename):
            with open(filename, 'w', encoding = 'utf-8') as f:
                f.write(header + '\n')
                
        
        if ser.in_waiting > 0:
            ser.reset_input_buffer()
        
        num_collected = 0
        
        while (datetime.now() - start_time) <= timedelta(seconds = seconds, minutes = minutes, hours = hours):
            if ser.in_waiting > 0:
                try:
                    line = ser.readline().decode('utf-8').strip()
                except:
                    print("DECODE FAILED")
                    continue
                line = line + f',{datetime.now()}'
                if not line.startswith("CSI_DATA"):
                    continue
                with open(filename, 'a', newline = '', encoding = 'utf-8') as f:
                    f.write(line + '\n')       
        
                num_collected += 1

                if num_collected % 5 == 0:
                    with open(temp_file_path, 'w', newline = '', encoding = 'utf-8') as f:
                        f.write(header + '\n')
                        f.write(line + '\n')  
                
                if num_collected % 100 == 0:
                    print(f"{datetime.now()} Number of CSI data collected : {num_collected}")
        os.remove(temp_file_path)
        
        print(f"Total : {num_collected}")
    

In [ ]:
PORT = "COM7"
BAUDRATE = "921600"
FILEPATH = "csi_data.csv"


reset_esp32(PORT, BAUDRATE)
collect_csi_data(FILEPATH, PORT, BAUDRATE)

ESP32 reset via DTR/RTS on port COM7
2025-03-01 09:37:29.578748 Number of CSI data collected : 100
2025-03-01 09:37:32.780743 Number of CSI data collected : 200
2025-03-01 09:37:36.017797 Number of CSI data collected : 300
2025-03-01 09:37:39.313769 Number of CSI data collected : 400
2025-03-01 09:37:42.581467 Number of CSI data collected : 500
2025-03-01 09:37:45.886396 Number of CSI data collected : 600
2025-03-01 09:37:49.154686 Number of CSI data collected : 700
2025-03-01 09:37:52.356102 Number of CSI data collected : 800
2025-03-01 09:37:55.650394 Number of CSI data collected : 900


In [1]:
import multiprocessing

In [2]:
res_queue = multiprocessing.Queue()
res_queue.put(7)

In [3]:
res_queue.put(8)